# Analyse trained models

In [ ]:
# Import packages
import torch
import numpy as np

import boltzgen as bg

from matplotlib import pyplot as plt

In [ ]:
# Specify checkpoint root
checkpoint_root = '/scratch2/vs488/flow/alanine_dipeptide/models/snf_01/'

In [ ]:
# Load config
config = bg.utils.get_config(checkpoint_root + 'config/bm.yaml')

In [ ]:
# Load data
training_data = bg.utils.load_traj('/scratch2/vs488/flow/alanine_dipeptide/trajectory/aldp100000.h5')

In [ ]:
# Setup model
model = bg.BoltzmannGenerator(config)

# Move model on GPU if available
enable_cuda = True
device = torch.device('cuda' if torch.cuda.is_available() and enable_cuda else 'cpu')
model = model.to(device)
model = model.double()

In [ ]:
# Plot loss
loss = np.loadtxt(checkpoint_root + 'log/loss.csv')
plt.figure(figsize=(15, 10))
plt.plot(loss)
plt.ylim(-310, -290)
plt.show()

In [ ]:
# Load checkpoint
model.load(checkpoint_root + 'checkpoints/bm_snf_model_04500.pt')

In [ ]:
import normflow as nf
dist = bg.distributions.TransformedBoltzmann(model.sim.context, 1000, energy_cut=1e10,
                                             energy_max=1e20, transform=model.flows[-1])
proposal = nf.distributions.DiagGaussianProposal((60,), 0.1 * np.ones(60))
mcmc = nf.flows.MetropolisHastings(dist, proposal, 100)
mcmc = mcmc.double().to('cuda')

In [ ]:
model.eval()

z_np = np.zeros((0, 60))
log_p_np = np.zeros((0,))
log_p_t_np = np.zeros((0,))
log_q_np = np.zeros((0,))
z_mcmc_np = np.zeros((0, 60))
log_p_mcmc_np = np.zeros((0,))
log_p_t_mcmc_np = np.zeros((0,))
for i in range(5):
    z, log_q = model.sample(1000)
    log_p = model.p.log_prob(z)
    z, _ = model.flows[-1].inverse(z)
    log_p_t = mcmc.dist.log_prob(z)
    z_np_ = z.cpu().data.numpy()
    log_p_np_ = log_p.cpu().data.numpy()
    log_p_t_np_ = log_p_t.cpu().data.numpy()
    log_q_np_ = log_q.cpu().data.numpy()
    z_np = np.concatenate((z_np, z_np_))
    log_p_np = np.concatenate((log_p_np, log_p_np_))
    log_p_t_np = np.concatenate((log_p_t_np, log_p_t_np_))
    log_q_np = np.concatenate((log_q_np, log_q_np_))

    z, _ = mcmc(z)
    z_mcmc_np_ = z.cpu().data.numpy()
    log_p_t = mcmc.dist.log_prob(z)
    z, _ = model.flows[-1](z)
    log_p = model.p.log_prob(z)
    log_p_mcmc_np_ = log_p.cpu().data.numpy()
    log_p_t_mcmc_np_ = log_p_t.cpu().data.numpy()
    z_mcmc_np = np.concatenate((z_mcmc_np, z_mcmc_np_))
    log_p_mcmc_np = np.concatenate((log_p_mcmc_np, log_p_mcmc_np_))
    log_p_t_mcmc_np = np.concatenate((log_p_t_mcmc_np, log_p_t_mcmc_np_))


z_d = training_data[::20].double().to(device)
log_p_d = model.p.log_prob(z_d)
z_d, _ = model.flows[-1].inverse(z_d)
log_p_t_d =  mcmc.dist.log_prob(z_d)
z_d_np = z_d.cpu().data.numpy()

log_p_d_np = log_p_d.cpu().data.numpy()
log_p_t_d_np = log_p_t_d.cpu().data.numpy()

In [ ]:
plt.hist(log_p_d_np, bins=100, alpha=0.5, label='data', range=[-70, 5])
plt.hist(log_p_np, bins=100, alpha=0.5, label='samples', range=[-70, 5])
plt.hist(log_p_mcmc_np, bins=100, alpha=0.5, label='mcmc', range=[-70, 5])
plt.legend(loc='upper right')
plt.show()

In [ ]:
plt.hist(log_p_t_d_np, bins=100, alpha=0.5, label='data', range=[-450, -360])
plt.hist(log_p_t_np, bins=100, alpha=0.5, label='samples', range=[-450, -360])
plt.hist(log_p_t_mcmc_np, bins=100, alpha=0.5, label='mcmc', range=[-450, -360])
plt.legend(loc='upper right')
plt.show()

In [ ]:
for i in range(60):
    print(i)
    plt.hist(z_d_np[:, i], bins=100, alpha=0.5, label='data', range=[-3.1, 3.1])
    plt.hist(z_np[:, i], bins=100, alpha=0.5, label='samples', range=[-3.1, 3.1])
    plt.hist(z_mcmc_np[:, i], bins=100, alpha=0.5, label='mcmc', range=[-3.1, 3.1])
    plt.legend(loc='upper right')
    plt.show()